# Creating your project and building the hydrological model

## Introduction

Notebook for the workflow **Snakefile_historical_hydrology**: 
1.  creates a Wflow model based on global data for the selected region, 
2.  runs the model for a selection of forcing datasets for the historical period, 
3.  evaluates the model performance using observed streamflow data (if provided) and snow cover extent (if provided).

In [2]:
# Import some python libraries we will need to run this notebook
import os
from IPython import display

In [3]:
# A helper function
from subprocess import Popen, PIPE, STDOUT

from IPython.core.magic import register_line_magic


@register_line_magic
def runrealcmd(command):
    with Popen(
        command, stdout=PIPE, shell=True, stderr=STDOUT, bufsize=1, close_fds=True
    ) as process:
        for line in iter(process.stdout.readline, b""):
            print(line.rstrip().decode("utf-8"))

## Defining your project region and properties

First thing is to prepare all the options for your project, this is done using the configuration or settings file below

In [4]:
# Add here the path where you have installed the blueearth-cst tool
cst_path = r'c:\Users\bouaziz\repos\blueearth_cst'
os.chdir(cst_path)

To create your model for any given (sub)basin, you will need to provide:

- for a basin to the sea, any point in the basin [x, y]
- for a subbasin (tributary or part of a basin) the location of the subbasin outlet [xout, yout]
- a bounding box [xmin, ymin, xmax, ymax]
- a geometry file of your region of interest (eg, myregion.shp)

For more details on the exact syntax, please have a look at:
https://deltares.github.io/hydromt/stable/user_guide/model_region.html#hydrographic-region 

The coordinates should be in EPSG 4326.
For points, you can use Google Maps to locate your coordinates.
For bounding box or point coordinates, you can also copy-paste locations from this website: http://bboxfinder.com/

In [7]:
%%writefile ./config/my-project-settings.yml

########################################################################################
#################################### General settings ##################################

# Path to where the wflow model will be stored (include the project name)
project_dir: tests/test_project

# hydromt "region" for building the model, coordinates are in EPSG 4326
# for more info on how to fill this, you can see the hydromt documentation: 
# https://deltares.github.io/hydromt/latest/user_guide/cli.html#region-options
# For wflow please use either 'basin' or 'subbasin' keywords
model_region: "{'subbasin': [12.3758, 46.4076], 'uparea': 70}"

# Path to the data catalog to use (can be different for Linux/Windows/data stored on P drive or other)
# NB: should be ordered from most recent to use last! 
data_sources: 
  - artifact_data
  - tests/data/tests_data_catalog.yml

# Info on the historical climate data is used in the historic model run + reference data for the weather generator
# Time horizon of the historical period
starttime: "2000-01-01T00:00:00"
endtime: "2010-12-31T00:00:00"

# Path to the data catalog to use (can be different for Linux/Windows/data stored on P drive or other)
data_sources_climate: config/cmip6_data.yml

# Historical gridded climate data sources (name as available in the data_sources catalog file). Either [era5, chirps_global, chirps, eobs].
clim_historical: 
  - era5
  - chirps_global

clim_historical_colors:
  - blue
  - green

# List of wflow output variables to save
# Supported variables: ['river discharge', 'precipitation', 'overland flow', 'actual evapotranspiration', 'groundwater recharge', 'snow']
# Comment out to disable saving
# Scalar csv
wflow_outvars: ['river discharge', 'precipitation', 'overland flow', 'actual evapotranspiration', 'groundwater recharge', 'snow']
# Gridded netcdf
wflow_outvars_gridded: ['snow']

###########################################################################################
#################################### Model building #######################################

# Grid resolution of the model in degrees
model_resolution: 0.0166666
# Path to model build configuration
model_build_config: "tests/wflow_build_model.yml"
waterbodies_config: "tests/wflow_update_waterbodies.yml"

# If available, please put here the path to the available observation stations locations file
# csv file containing four columns: station_ID, name, x, y
output_locations: "tests/data/observations/discharge-locations-grdc.csv" # If set to None, will default to the wflow outlets
# Path to the file with observed discharge timeseries per station 
observations_timeseries: "tests/data/observations/observations-discharge-grdc.csv" 

# Additional plot of budyko
plot_budyko: True

Overwriting ./config/my-project-settings.yml


## Running the model building workflow

The first line is to create an image of our workflow with snakemake to verify that all our settings are well taken into account.

In [ ]:
! snakemake -s Snakefile_model_creation --configfile config/my-project-settings.yml  --dag | dot -Tpng > dag_model_building.png

In [ ]:
display.Image("./dag_model_building.png")

Now we will unlock your directory in order to save the future results of the workflow. This step is most of the time not necessary.

In [ ]:
! snakemake --unlock -s Snakefile_model_creation --configfile config/my-project-settings.yml

And finally let's run our worflow:

In [ ]:
%runrealcmd snakemake all -c 1 -s Snakefile_model_creation --configfile config/my-project-settings.yml

## Results

First check that snake did run everything.

In [ ]:
! snakemake -s Snakefile_model_creation --configfile config/my-project-settings.yml  --dag | dot -Tpng > dag_model_building.png

In [ ]:
display.Image("./dag_model_building.png")

Let's see which files where created in our output folder.

In [ ]:
root = "examples/myModel"
for path, _, files in os.walk(root):
    print(path)
    for name in files:
        if name.endswith(".xml"):
            continue
        print(f" - {name}")

In [ ]:
display.Image("./examples/myModel/plots/wflow_model_performance/basin_area.png")

In [ ]:
display.Image("./examples/myModel/plots/wflow_model_performance/hydro_wflow_1.png")